# Creating Features

Importing Libraries

In [2]:
import os
import numpy as np
import pandas as pd
from pymatgen.core import Structure

# Helper Functions to keep notebook clean
import functions as func

Define file and folder paths

In [3]:
PATH = os.getcwd() + "/"

data_pickle_CTE = PATH + 'DATA/pickle/data_cte.pkl'

Load dataset from previous step

In [4]:
df_CTE = pd.read_pickle(data_pickle_CTE)

In [5]:
from matminer.featurizers.structure.composite import JarvisCFID
from matminer.featurizers.structure.bonding import StructuralHeterogeneity
from matminer.featurizers.structure.sites import SiteStatsFingerprint

jarvis = JarvisCFID()
sthet = StructuralHeterogeneity()
ssf = SiteStatsFingerprint.from_preset("LocalPropertyDifference_ward-prb-2017")
#ewald = EwaldSiteEnergy()
#soap = SOAP(rcut=5, nmax=8, lmax=8)

In [6]:
featurizers_list = [jarvis, ssf]

data_numpy_jarvis = 'DATA/feat/data_Jarvis_CFID.npy'
data_numpy_sthet = 'DATA/feat/data_Structural_Heterogenity.npy'
data_numpy_ssf = 'DATA/feat/data_Site_Stats_Fingerprint.npy'

Create a list containing featurized arrays

In [7]:

ff = []
for feat in featurizers_list:
    n = 0
    list_feat = []
    x, x_tot = 0, len(df_CTE)
    for i in range(len(df_CTE)):
        func.progress_bar(x, x_tot)
        x += 1
        try:
            df_temp = pd.DataFrame(df_CTE[['@module','@class','charge','lattice','sites']].iloc[i]).transpose()
            struc_dict = df_temp.to_dict(orient='records')
            structure = Structure.from_dict(struc_dict[0])
            feature = feat.featurize(structure)

            prefix = np.array(df_CTE[['idx','dir NTE','dir NTE large','vol NTE']].iloc[i])
            feat_ab = np.concatenate((prefix, feature), axis=0)
            list_feat.append(feat_ab)
        except:
            n += 1
    print("Failed: " + str(n))
    final_feat = np.stack((list_feat[j] for j in range(len(list_feat))))
    ff.append(final_feat)



In [ ]:

np.save(data_numpy_sthet, final_feat)